In [1]:
!pip install psycopg2

In [2]:
import psycopg2

In [3]:
try:
    conn = psycopg2.connect("host = localhost dbname = postgres user = postgres password = admin")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

## Use the connection to get a cursor that can be used to execute

In [4]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print('Error: Could not get curser to the Database')
    print(e)

In [5]:
conn.set_session(autocommit = True)

In [6]:
try:
    cur.execute("create database songs")
except psycopg2.Error as e:
    print(e)

In [7]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

try:
    conn = psycopg2.connect("host = localhost dbname = songs user = postgres password = admin")
except psycopg2.Error as e:
    print("Error: Could not make a connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

In [8]:
conn.set_session(autocommit = True)

In [9]:
import pandas as pd
import requests
import json
from datetime import datetime
import datetime
from sqlalchemy import create_engine
import time

In [28]:
TOKEN = "BQA_R05leH1e-vwuAqYm7dOpJIlhj0uuKDHjEqOLc39AL94CixkDvhttltJDwwKf3dliWCdQXC7sG_oith315RDvpfZDe8EBjLHi9xKe4oHRCkBzbPJzVMEhDYmOklJT8ajXUd7SBvPOsRwX92SdTL-BxpnyXcgDhAJ3bM0vbZ0QSE-J5nxfsK1JhUHu5eTLygkpjSxNfCpk" # your Spotify API token


In [29]:
'''
def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
        return False 

    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check is violated")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")

    
    # Check that all timestamps are of yesterday's date
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    timestamps = df["timestamp"].tolist()
    for timestamp in timestamps:
        if datetime.datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
            raise Exception("At least one of the returned songs does not have a yesterday's timestamp")

    return True
    '''

if __name__ == "__main__":

    # Extract part of the ETL process
 
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }
    
    # Convert time to Unix timestamp in miliseconds      
    #today = datetime.datetime.now()
    #yesterday = today - datetime.timedelta(days=1)
    #yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    # Download all songs you've listened to "after yesterday", which means in the last 24 hours      
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played", headers = headers)

    data = r.json()

    
    

    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []

    
    # Extracting only the relevant bits of data from the json object      
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])
        
    # Prepare a dictionary in order to turn it into a pandas dataframe below       
    song_dict = {
        "song_name" : song_names,
        "artist_name": artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps
    }

    song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])


    #print(song_df)
    
    '''# Validate
    if check_if_valid_data(song_df):
        print("Data valid, proceed to Load stage")'''
    
    #song_df.to_csv('Spotify.csv')
    
    try:
        cur.execute('create table if not exists Spotify(song_name varchar, artist_name varchar, played_at varchar, timestamp varchar);')
    except psycopg2.Error as e:
        print(e)
    

In [30]:
insert = ("""insert into Spotify(
song_name,
artist_name,
played_at,
timestamp)
values(%s, %s, %s, %s)""")

In [31]:
for i, row in song_df.iterrows():
    cur.execute(insert, list(row))

In [32]:
conn.commit()